In [1]:
from datasets import load_dataset

dataset = load_dataset("dansbecker/hackernews_hiring_posts")

Found cached dataset parquet (/Users/busesibelkorkmaz/.cache/huggingface/datasets/dansbecker___parquet/dansbecker--hackernews_hiring_posts-bf0ef9ae958fdb08/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
first_10_rows = dataset["hiring"][slice(None, 10, None)]


In [7]:
first_10_rows

{'text': ['Scoot &#38; Doodle (SF/Remote)\nRaised $2.25M in funding and user engagement averages 30 minutes/session.\nLooking for  Rails, iOS, NodeJS devs.\nLearn more - <a href="http://blog.goodsense.io/jobs-and-projects" >http://blog.goodsense.io/jobs-and-projects</a>\nIf you want to talk about other projects contact me here - <a href="http://blog.goodsense.io/contact" >http://blog.goodsense.io/contact</a>',
  'Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP).\nFull list of positions: <a href="http://www.camptocamp.com/en/careers" >http://www.camptocamp.com/en/careers</a>',
  'Portland, OR/San Francisco - <a href="http://newrelic.com/about/jobs" >http://newrelic.com/about/jobs</a>\nNew Relic is growing and has several technical (and non-technical) positions.  If you\'ve 

In [2]:
dataset

DatasetDict({
    freelancer: Dataset({
        features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle'],
        num_rows: 15266
    })
    hiring: Dataset({
        features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle'],
        num_rows: 75974
    })
    wants_to_be_hired: Dataset({
        features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle'],
        num_rows: 20262
    })
})

In [3]:
dataset["hiring"]

Dataset({
    features: ['text', 'CommentTime', 'CommentAuthor', 'ParentTitle'],
    num_rows: 75974
})

In [4]:
hiring_df  = dataset["hiring"].to_pandas()

In [4]:
import re

def preprocess_text(row):
    text = str(row["text"])
    text = re.sub("\n", '', text)
    text = re.sub('"', '', text)

    first_step = re.sub(r'http\S+|www.\S+', '', text)
    second_step = re.sub(r'<a href=+|>+', '', first_step)

    return {"clean_text": second_step}

dataset["hiring"] = dataset["hiring"].map(preprocess_text, num_proc=8)


Loading cached processed dataset at /Users/busesibelkorkmaz/.cache/huggingface/datasets/dansbecker___parquet/dansbecker--hackernews_hiring_posts-bf0ef9ae958fdb08/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7/cache-19b3397aceccf5ac_*_of_00008.arrow


In [5]:
hiring_df.loc[0]["text"]

NameError: name 'hiring_df' is not defined

In [7]:
hiring_df.loc[1]["text"]

'Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP).\nFull list of positions: <a href="http://www.camptocamp.com/en/careers" >http://www.camptocamp.com/en/careers</a>'

In [8]:
dataset["wants_to_be_hired"].to_pandas()

,text,CommentTime,CommentAuthor,ParentTitle
0,"Location: Pittsburgh, PA\nRemote: OK\nWilling ...",2014-08-01 15:20:48 UTC,mtdavis,Ask HN: Who wants to be hired? (August 2014)
1,"\n Location: East coast, US\n Remote: Yes\n ...",2021-09-03 03:15:41 UTC,kvg8C091,Ask HN: Who wants to be hired? (September 2021)
2,Location: Germany\nRemote: yes (but would pref...,2019-02-02 14:52:35 UTC,kleinfreund,Ask HN: Who wants to be hired? (February 2019)
3,"Location: Birmingham, UK\nRemote: Preferred\nW...",2021-08-04 13:35:08 UTC,Jefro118,Ask HN: Who wants to be hired? (August 2021)
4,Location: US East Coast\nWilling to Relocate: ...,2019-03-01 19:55:30 UTC,LongTermBond007,Ask HN: Who wants to be hired? (March 2019)
...,...,...,...,...
20257,"Location: Chicago, IL\nRemote: Will consider\n...",2017-07-08 09:05:06 UTC,rwesty,Ask HN: Who wants to be hired? (July 2017)
20258,"Location: Rome, Italy\nRemote: YES\nWilling to...",2017-07-08 12:39:13 UTC,eloquentbit,Ask HN: Who wants to be hired? (July 2017)
20259,"Location: Montreal, Quebec (Canada)\nRemote: N...",2017-07-05 18:12:05 UTC,ajgaba,Ask HN: Who wants to be hired? (July 2017)
20260,"Location: Seattle, WA\nRemote: yes/exclusively...",2017-07-03 16:33:10 UTC,fuzzy-logic,Ask HN: Who wants to be hired? (July 2017)


In [9]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define your prompt
prompt = "translate English to French: I love programming"

# Encode the prompt
inputs = tokenizer.encode(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs, max_length=40, num_return_sequences=1, temperature=0.7)

# Decode the output
decoded_output = tokenizer.decode(outputs[0])

print(decoded_output)

/Users/busesibelkorkmaz/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<pad> J'aime la programmation</s>


In [10]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Define your prompt
prompt = "Parse this job description as bullet points of 'location(s)' and 'technologies': 'Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP).'"

# Encode the prompt
inputs = tokenizer.encode(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(inputs, max_length=200, num_return_sequences=1, temperature=0.7)

# Decode the output
decoded_output = tokenizer.decode(outputs[0])

print(decoded_output)

<pad> 'Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business development': 'All open source. Javascript (OpenLayers, Ext.js, etc.) and Python (Pylons, SQLAlchemy, etc.). Business development is Python (OpenERP).'</s>


In [11]:
# from transformers import pipeline

# # Specify the models
# models = ['dslim/bert-base-NER']

# # Input text
# text = "Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP)"

# # Process the text with each model
# for model in models:
#     ner = pipeline("ner", model=model)
#     entities = ner(text)
#     print(f"Entities recognized by {model}:")
#     for entity in entities:
#         print(f"{entity['entity']}: {entity['word']}")
#     print("\n")


In [12]:
from transformers import AutoTokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Prompt for extracting locations
prompt = "What are the locations mentioned in the following text: Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business\nAll open source.\nGeospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).\nBusiness development is Python (OpenERP) ?"

# Tokenize the input
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generate a response
outputs = model.generate(input_ids, min_length=20)

# Decode the output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(decoded_output)


/Users/busesibelkorkmaz/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:156: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/Users/busesibelkorkmaz/miniconda3/lib/python3.10/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.m

True - False - False - True - True -


In [13]:
from transformers import pipeline

# Initialize the question-answering pipeline
model_name = "deepset/roberta-base-squad2"
nlp = pipeline("question-answering", model=model_name, tokenizer=model_name, max_length=512)

# The context from which the model will extract information
context = """
Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business
All open source.
Geospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).
Business development is Python (OpenERP).
"""

# Questions you want the model to answer
questions = [
    "What are the locations mentioned in the text?",
    "What technologies are mentioned in the text?",
    "What technical skills are mentioned in the text?",
]

# Ask each question
for question in questions:
    result = nlp(question=question, context=context)
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")


Question: What are the locations mentioned in the text?
Answer: 
Chambéry, France and Lausanne, Switzerland
Question: What technologies are mentioned in the text?
Answer: Pylons, SQLAlchemy, etc.).
Question: What technical skills are mentioned in the text?
Answer: Python and Javascript


In [14]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

# The context from which the model will extract information
context = """
Chambéry, France and Lausanne, Switzerland - Python and Javascript devs - geospatial and business
All open source.
Geospatial development is primarily Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.).
Business development is Python (OpenERP).
"""

# Questions you want the model to answer
questions = [
    "What are the locations mentioned in the text?",
    "What technologies and libraries are mentioned in the text?",
    # "What technical skills are mentioned in the text?",
]

# Iterate over each question
for question in questions:
    # Concatenate the question and the context
    input_text = "question: " + question + " context: " + context

    # Encode the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Pass the input to the model and get the predicted answer
    output = model.generate(input_ids, max_length=128)
    answer = tokenizer.decode(output[0])

    # Print the question and the predicted answer
    print("Question:", question)
    print("Answer:", answer)


/Users/busesibelkorkmaz/miniconda3/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:164: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


Question: What are the locations mentioned in the text?
Answer: <pad> Chambéry, France and Lausanne, Switzerland</s>
Question: What technologies and libraries are mentioned in the text?
Answer: <pad> Javascript (OpenLayers, Ext.js) and Python (Pylons, SQLAlchemy, etc.)</s>


In [15]:
hiring_df.dropna(inplace=True)

In [16]:
len(hiring_df)

73445

In [17]:
def not_none(example):
    return example['text'] is not None

dataset["hiring"] = dataset["hiring"].filter(not_none)

Filter:   0%|          | 0/75974 [00:00<?, ? examples/s]

In [27]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import pipeline

def parse_job_description(
    row, 
    question:str,
    counter: int,
    ) -> str:
    
    counter +=1 
    if counter % 100 == 0:
        print(f"{question} progress %{100*(counter/len(hiring_df))}")

    # # Initialize the tokenizer and model
    # tokenizer = T5Tokenizer.from_pretrained("t5-base", model_max_length=512)
    # model = T5ForConditionalGeneration.from_pretrained("t5-base")

    # # The context from which the model will extract information
    # context = row["text"]

    # # Questions you want the model to answer
    # questions = {
    #     "loc": "What are the locations mentioned in the text?",
    #     "tech": "What technologies and libraries are mentioned in the text?",
    #     # "What technical skills are mentioned in the text?",
    # }

    # # Iterate over each question
    # # Concatenate the question and the context
    # input_text = "question: " + questions[question] + " context: " + context

    # # Encode the input text
    # input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # # Pass the input to the model and get the predicted answer
    # output = model.generate(input_ids, max_length=128)
    # answer = tokenizer.decode(output[0])


    # Initialize the question-answering pipeline
    model_name = "deepset/roberta-base-squad2"
    nlp = pipeline("question-answering", model=model_name, tokenizer=model_name, max_length=512)

    # The context from which the model will extract information
    context = row["text"]

    # Questions you want the model to answer
    questions = {
        "loc": "What are the locations mentioned in the text?",
        "tech": "What technologies and libraries are mentioned in the text?",
        # "What technical skills are mentioned in the text?",
    }

    # Ask each question
    result = nlp(question=questions[question], context=context)
    print(result["answer"])
    # print(f"Question: {question}")
    # print(f"Answer: {result['answer']}")

    # Print the question and the predicted answer
    # print("Question:", question)
    # print("Answer:", answer)
    if question ==  "loc":
        key = "location"
    else:
        key = "technologies"

    return {key: result["answer"]}

counter = 0
# dataset["hiring"] = dataset["hiring"].map(preprocess_text, num_proc=8)

dataset["hiring"] = dataset["hiring"].map(lambda x: parse_job_description(x, "loc", counter), num_proc=4)
counter = 0
# hiring_df["technologies"] = hiring_df["clean_text"].apply(lambda x: parse_job_description(x, "tech", counter))

Map (num_proc=4):   0%|          | 0/73445 [00:00<?, ? examples/s]

TimeoutError: 

In [6]:
dataset["hiring"].save_to_disk("processed_hiring")

Saving the dataset (0/1 shards):   0%|          | 0/75974 [00:00<?, ? examples/s]